In [1]:
import tensorflow as tf
import sys

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.apps as app

In [2]:
config = helper.ConfigParser(sys.argv, 'inference')

Reading default config file in: C:\Users\yhu\Git\tutorials2019\weakly\config_demo.ini.

[Network]: network_type: local
[Inference]: file_model_saved: C:\Users\yhu/git/label-reg/data/model.ckpt
[Inference]: dir_moving_image: C:\Users\yhu/git/label-reg/data/test/mr_images
[Inference]: dir_fixed_image: C:\Users\yhu/git/label-reg/data/test/us_images
[Inference]: dir_save: C:\Users\yhu/git/label-reg/data/
[Inference]: dir_moving_label: C:\Users\yhu/git/label-reg/data/test/mr_labels
[Inference]: dir_fixed_label: C:\Users\yhu/git/label-reg/data/test/us_labels



In [3]:
reader_moving_image, reader_fixed_image, _, _ = helper.get_data_readers(config['Inference']['dir_moving_image'],
                                                                        config['Inference']['dir_fixed_image'])


In [4]:
ph_moving_image = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
ph_fixed_image = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])


In [5]:
reg_net = network.build_network(network_type=config['Network']['network_type'],
                                minibatch_size=reader_moving_image.num_data,
                                image_moving=ph_moving_image,
                                image_fixed=ph_fixed_image)

Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, config['Inference']['file_model_saved'])

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\yhu/git/label-reg/data/model.ckpt


In [7]:
testFeed = {ph_moving_image: reader_moving_image.get_data(),
            ph_fixed_image: reader_fixed_image.get_data()}
ddf = sess.run(reg_net.ddf, feed_dict=testFeed)
helper.write_images(ddf, config['Inference']['dir_save'], 'ddf')

In [8]:
warped_images = app.warp_volumes_by_ddf(reader_moving_image.get_data(), ddf)
helper.write_images(warped_images, config['Inference']['dir_save'], 'warped_image')

In [9]:
warped_images = app.warp_volumes_by_ddf(reader_moving_image.get_data(), ddf)
helper.write_images(warped_images, config['Inference']['dir_save'], 'warped_image')